In [2]:
import backtrader as bt
from pykrx import stock
import pandas as pd

AttributeError: module 'wrapt' has no attribute 'AdapterFactory'

### 볼린저 밴드(20, 2)

#### 1. 볼린저밴드 하단선 매수, 중단서 매도

In [20]:
class MyStrategy(bt.Strategy): # bt.Strategy를 상속함

    # 볼린저 밴드에 사용할 이동평균일 수와 표준편차에 곱할 상수를 정의한다.
    params = (
        ("period", 20),
        ("devfacter",2),
        ("debug",False)
    )
    
    # 프롬프트에 매수 or 매도, 매수매도 가격, 개수를 출력한다.
    def log(self,txt, dt=None):
        ''' Logging function fot this strategy '''
        dt = dt or self.data.datetime[0]
        if isinstance(dt,float):
            dt = bt.num2date(dt)
        print("%s, %s"% (dt.isoformat(), txt))
    
    
    def __init__(self):
        
        # 볼린저 밴드 지표를 가져온다
        self.boll = bt.indicators.BollingerBands(period = self.p.period, 
                                                plot =True)
    def next(self):
        global size
        if not self.position: # 매수한 종목이 없다면
            if self.data.low[0] < self.boll.lines.bot[0]: # 저가 < 볼린저 밴드 하한선이면(종가로 수정 요망)
                bottom = self.boll.lines.bot[0]             
                size = int(self.broker.getcash() /bottom) # size는 매수 또는 매도 개수로 현재 금액에서 타겟가격으로 나누어준다.
                                                          # 볼린저 밴드 하한선에서 구매시 최대 구매 가능 개수
                self.buy(price=bottom, size=size)         # 매수 size 즉 구매 개수 설정
                self.log("BUY CREATE, %.2f" % (bottom))   # 프롬프트에 매수 로그 출력
                print(size,"EA")
        else:
            if self.data.high[0] > self.boll.lines.mid[0]:              # 고가 > 볼린저 밴드 중간선이면(종가로 수정 요망) 
                self.sell(price=self.boll.lines.mid[0], size=size)      # 매도: 20일 이동 평균선 = 볼린저 밴드 중간선
                self.log('SELL CREATE, %2.f' % (self.boll.lines.mid[0]))# 프롬프트에 매도 로그 출력
                print(size, 'EA')

In [21]:
size = 0 
stock_name = "KODEX 200"
stock_from = "20171125"
stock_to = "20211125"

stock_list = pd.DataFrame({'종목코드':stock.get_etf_ticker_list(stock_to)})
stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_etf_ticker_name(x))
stock_list.head()

ticker = stock_list.loc[stock_list['종목명']==stock_name, '종목코드']
df = stock.get_etf_ohlcv_by_date(fromdate=stock_from, todate=stock_to, ticker=ticker)
df = df.drop(['NAV','거래대금','기초지수'], axis=1)
df = df.rename(columns={'시가':'open', '고가':'high', '저가':'low', '종가':'close', '거래량':'volume'})

df["open"]=df["open"].apply(pd.to_numeric,errors="coerce")
df["high"]=df["high"].apply(pd.to_numeric,errors="coerce")
df["low"]=df["low"].apply(pd.to_numeric,errors="coerce")
df["close"]=df["close"].apply(pd.to_numeric,errors="coerce")
df["volume"]=df["volume"].apply(pd.to_numeric,errors="coerce")

In [27]:
data = bt.feeds.PandasData(dataname=df)
cerebro = bt.Cerebro() # Cerebro 엔진 인스턴스 생성
cerebro.broker.setcash(100000) # 초기자금
cerebro.broker.setcommission(0.00015) # 수수료

cerebro.adddata(data) #data feed를 추가
cerebro.addstrategy(MyStrategy) #Add the trading strategy
cerebro.run()
cerebro.plot(style='candlestick',barup='red',bardown="blue",xtight=True,yTight=True,grid=True) # plot


2017-12-22T00:00:00, BUY CREATE, 32061.07
3 EA
2017-12-28T00:00:00, SELL CREATE, 32524
3 EA
2018-02-05T00:00:00, BUY CREATE, 32668.90
3 EA
2018-02-27T00:00:00, SELL CREATE, 32257
3 EA
2018-04-04T00:00:00, BUY CREATE, 31324.06
3 EA
2018-04-13T00:00:00, SELL CREATE, 31939
3 EA
2018-05-08T00:00:00, BUY CREATE, 31545.43
3 EA
2018-05-23T00:00:00, SELL CREATE, 31831
3 EA
2018-05-30T00:00:00, BUY CREATE, 31181.13
3 EA
2018-06-07T00:00:00, SELL CREATE, 31591
3 EA
2018-06-18T00:00:00, BUY CREATE, 30790.44
3 EA
2018-07-13T00:00:00, SELL CREATE, 29916
3 EA
2018-08-02T00:00:00, BUY CREATE, 29426.52
3 EA
2018-08-06T00:00:00, SELL CREATE, 29719
3 EA
2018-08-10T00:00:00, BUY CREATE, 29423.20
3 EA
2018-08-23T00:00:00, SELL CREATE, 29467
3 EA
2018-10-10T00:00:00, BUY CREATE, 28824.93
3 EA
2018-11-07T00:00:00, SELL CREATE, 27234
3 EA
2018-12-10T00:00:00, BUY CREATE, 26518.53
3 EA
2018-12-13T00:00:00, SELL CREATE, 27022
3 EA
2018-12-26T00:00:00, BUY CREATE, 26080.17
4 EA
2019-01-03T00:00:00, BUY CREATE, 

ImportError: cannot import name 'warnings' from 'matplotlib.dates' (c:\Users\dgjja\anaconda3\lib\site-packages\matplotlib\dates.py)